In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import scipy
import json
import ast
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
meanMeds = pd.read_csv('meanMedsFiltered.csv')

In [ ]:
# Z-test for statiscical significance of median increase
for year in range(2019, 2024):
    meanMeds[f'z-MedIncrease{year}'] = stats.zscore(meanMeds[f'medIncrease{year}'])

    # Find p-value from z-score, multiplied by 2 for 2 tailed test
    meanMeds[f'p-MedIncrease{year}'] = stats.norm.sf(np.abs(meanMeds[f'z-MedIncrease{year}']))*2




In [ ]:
def chi2(year):
    pvalues = []
    for i, row in meanMeds.iterrows():
        # get dicts from the string they are stored as
        dict1 = ast.literal_eval(row[f'race{year-1}'])
        dict2 = ast.literal_eval(row[f'race{year}'])

        # get a set of all keys
        keys = set(dict1.keys()).union(dict2.keys())

        dict1F = {key: dict1.get(key, 0) for key in keys}
        dict2F = {key: dict2.get(key, 0) for key in keys}


        table = [[dict1F[k], dict2F[k]] for k in keys]

        stat, p, dof, expected = stats.chi2_contingency(table)
        pvalues.append(p)
    
    return pvalues

for i in range(2019,2024):
    meanMeds[f'chi{i}'] = chi2(i)

In [ ]:
print(meanMeds['medIncrease2023'].describe())

In [ ]:
for i in range(2019, 2024):
    sns.regplot(x=meanMeds[f'chi{i}'], y=meanMeds[f'p-MedIncrease{i}'], line_kws={'color': 'red'})
    plt.show()

In [ ]:
for i in range(2019, 2024):
    sns.regplot(x=meanMeds[f'chi{i}'], y=meanMeds[f'medIncrease{i}'], fit_reg=True, line_kws={'color': 'red'})
    plt.show()

In [ ]:
czconnect = pd.read_csv('/Users/shleiferj/downloads/Citizen_Connect_-_County_data__live__20241105.csv')
# Filter for only commuting and demographic data
commuters = czconnect[czconnect['Category'] == 'Commuting']
demographics = czconnect[czconnect['Category'] == 'Demographics']
del czconnect

resPermits = pd.read_csv('/Users/shleiferj/downloads/Residential_Construction_Permits_by_County_915231567991157423.csv')

# years are separated by columns, so filter for only applicable years
dataRange = ['GEOID']
for year in range(2018, 2023):
    dataRange.append(f'ALL_PERMITS_{year}')
    dataRange.append(f'SINGLE_FAMILY_PERMITS_{year}')
    dataRange.append(f'ALL_MULTIFAMILY_PERMITS_{year}')
    dataRange.append(f'MULTIFAMILY_PERMITS_2_UNITS_{year}')
    dataRange.append(f'MULTIFAMILY_PERMITS_3_4_UNITS_{year}')
    dataRange.append(f'MULTIFAMILY_PERMITS_5_OR_MORE_UNITS_{year}')
# These two are labeled slightly differently, no idea why
dataRange[-2], dataRange[-1] = ['MULTIFAMILY_PERMITS_3_4_UNIT_2022', 'MULTIFAMILY_PERMITS_5_OR_MORE_2022']
resPermits = resPermits[dataRange]
resPermits.index = resPermits['GEOID']


In [ ]:
temp = resPermits.copy()
for i in range(2019, 2023):
    temp[f'medincrease{i}'] = meanMeds[f'medIncrease{i}']
    sns.regplot(x=temp[f'SINGLE_FAMILY_PERMITS_{i}'], y=temp[f'medincrease{i}'])
    plt.show()


# FEMA, remove disaster areas (disaster decleration)

In [ ]:
df.pivot(index='patient', columns='obs', values='score')

commuters.pivot(index='GEOID', columns=['year', ])